# AWQ Quantization for dots.ocr Model

Using **llm-compressor** (vLLM's official successor to AutoAWQ)

**⚠️ BEFORE RUNNING - Verify GPU is enabled:**
1. Right panel → Settings → Accelerator → **GPU T4 x2**
2. Right panel → Settings → Internet → **On**
3. You should see "GPU T4 x2" in the top right corner

## Step 0: Verify GPU is Active

In [ ]:
# Run this FIRST to verify GPU
!nvidia-smi

In [ ]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

if torch.cuda.is_available():
    print(f"GPU count: {torch.cuda.device_count()}")
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print("\n✓ GPU is ready!")
else:
    print("\n❌ NO GPU DETECTED!")
    print("\nTo fix:")
    print("1. Go to Settings (right panel)")
    print("2. Under 'Accelerator', select 'GPU T4 x2'")
    print("3. Wait for session to restart")
    print("4. Re-run this cell")
    raise RuntimeError("GPU not available. Enable it in Kaggle settings.")

## Step 1: Install Dependencies

In [ ]:
!pip install llmcompressor --quiet
!pip install accelerate --quiet

## Step 2: Download Model

In [ ]:
from huggingface_hub import snapshot_download
import os

MODEL_ID = "rednote-hilab/dots.ocr"
MODEL_DIR = "/kaggle/working/dots_ocr_original"
OUTPUT_DIR = "/kaggle/working/dots_ocr_awq_4bit"

print(f"Downloading {MODEL_ID}...")
snapshot_download(repo_id=MODEL_ID, local_dir=MODEL_DIR, local_dir_use_symlinks=False)
print(f"Downloaded to {MODEL_DIR}")

In [ ]:
# Show files
for f in os.listdir(MODEL_DIR):
    path = os.path.join(MODEL_DIR, f)
    if os.path.isfile(path):
        print(f"{f}: {os.path.getsize(path)/1024**2:.1f} MB")

## Step 2.5: Patch DotsVLProcessor for Newer Transformers

The DotsVLProcessor class needs to be patched to work with newer transformers that require video_processor.

In [ ]:
# Patch the DotsVLProcessor class to handle video_processor requirement
import re

config_path = os.path.join(MODEL_DIR, "configuration_dots.py")
with open(config_path, 'r') as f:
    content = f.read()

# Replace the DotsVLProcessor class definition
old_class_pattern = r'class DotsVLProcessor\(Qwen2_5_VLProcessor\):.*?(?=\nclass |\Z)'
new_class = '''class DotsVLProcessor(Qwen2_5_VLProcessor):
    def __init__(self, image_processor=None, tokenizer=None, video_processor=None, chat_template=None, **kwargs):
        # If video_processor is None, create one to satisfy newer transformers requirement
        if video_processor is None:
            try:
                from transformers import Qwen2VLVideoProcessor
                video_processor = Qwen2VLVideoProcessor()
            except (ImportError, Exception):
                # Create a minimal video processor if the import fails
                from transformers.models.qwen2_vl.video_processing_qwen2_vl import Qwen2VLVideoProcessor
                video_processor = Qwen2VLVideoProcessor()
        
        super().__init__(image_processor, tokenizer, video_processor=video_processor, chat_template=chat_template, **kwargs)
        self.image_token = "<|imgpad|>" if not hasattr(tokenizer, "image_token") else tokenizer.image_token
        self.image_token_id = 151665 if not hasattr(tokenizer, "image_token_id") else tokenizer.image_token_id
'''

new_content = re.sub(old_class_pattern, new_class, content, flags=re.DOTALL)

with open(config_path, 'w') as f:
    f.write(new_content)

print("✓ Patched DotsVLProcessor to handle video_processor requirement")

## Step 3: Load Model, Tokenizer, and Processor

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoProcessor, AutoImageProcessor
import gc

# Clear GPU memory
torch.cuda.empty_cache()
gc.collect()

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

In [ ]:
# Load tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
print(f"Tokenizer loaded. Vocab size: {tokenizer.vocab_size}")

In [ ]:
# Load processor with manual video_processor construction
print("Loading processor...")

# Clear any cached modules to pick up the patched version
import sys
modules_to_remove = [key for key in sys.modules.keys() if 'dots_ocr_original' in key]
for mod in modules_to_remove:
    del sys.modules[mod]

processor = AutoProcessor.from_pretrained(MODEL_DIR, trust_remote_code=True)
print(f"Processor loaded: {type(processor).__name__}")

In [ ]:
# Load model on GPU
print("Loading model to GPU...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_DIR,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="cuda",  # Explicitly use CUDA
)
print(f"Model loaded! GPU mem: {torch.cuda.memory_allocated()/1024**3:.2f} GB")

## Step 4: Prepare Calibration Data

In [ ]:
from datasets import Dataset

# Calibration prompts
calibration_texts = [
    "What text is shown in this image?",
    "Please read and transcribe the text from this document.",
    "Extract all the text content from the provided image.",
    "Can you identify and read the text in this picture?",
    "Transcribe the handwritten text in this image.",
    "What does this document say?",
    "Read the printed text from this image.",
    "Please OCR this image and provide the text.",
    "Extract text content including numbers and special characters.",
    "Identify all readable text elements in this image.",
    "What is written on this page?",
    "Transcribe the following scanned document.",
    "Please read the text from this screenshot.",
    "Extract and format the text content visible in the image.",
    "Read all text including headers and footnotes.",
    "What are the words shown in this picture?",
    "Describe what you see in the image.",
    "List all words visible in this photograph.",
    "Convert the printed text to digital format.",
    "Read and output the document content.",
    "Extract text from this scanned page.",
    "What words appear in this image?",
    "Transcribe the visible text accurately.",
    "Identify all text elements in the picture.",
    "Read the content of this document image.",
    "Extract and list all readable text.",
    "What is the text content of this image?",
    "Please digitize the text in this scan.",
    "Convert image text to editable format.",
    "Read aloud what this document says.",
    "Extract every word from this image.",
    "What textual information is present?"
]

calibration_dataset = Dataset.from_dict({"text": calibration_texts})
print(f"Calibration dataset: {len(calibration_dataset)} samples")

In [ ]:
# Tokenize calibration data
def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True, max_length=512)

tokenized_dataset = calibration_dataset.map(tokenize_function, batched=True)
print("Calibration data tokenized!")

## Step 5: Quantize with AWQ

In [ ]:
from llmcompressor.modifiers.quantization import QuantizationModifier
from llmcompressor import oneshot

# AWQ-style quantization recipe
recipe = QuantizationModifier(
    targets="Linear",
    scheme="W4A16",  # 4-bit weights, 16-bit activations
    ignore=["lm_head"],  # Don't quantize the output layer
)

print("Quantization config:")
print("  - Scheme: W4A16 (4-bit weights)")
print("  - Target: Linear layers")
print("  - Ignored: lm_head")

In [ ]:
print("="*50)
print("Starting Quantization (15-30 mins)...")
print("="*50)

oneshot(
    model=model,
    processor=processor,  # Processor includes tokenizer - don't pass both!
    dataset=tokenized_dataset,
    recipe=recipe,
    output_dir=OUTPUT_DIR,
    num_calibration_samples=len(calibration_texts),
    save_compressed=True,
)

print("="*50)
print("Quantization Complete!")
print("="*50)

## Step 6: Save Processor and Config Files

In [ ]:
import shutil

# Save tokenizer and processor
tokenizer.save_pretrained(OUTPUT_DIR)
processor.save_pretrained(OUTPUT_DIR)
print("Saved tokenizer and processor")

# Copy extra config files (including the patched configuration_dots.py)
for f in ["preprocessor_config.json", "generation_config.json", "chat_template.jinja",
          "modeling_dots.py", "modeling_dots_vision.py", "configuration_dots.py",
          "image_processing_dots.py", "processing_dots.py"]:
    src = os.path.join(MODEL_DIR, f)
    if os.path.exists(src):
        shutil.copy2(src, os.path.join(OUTPUT_DIR, f))
        print(f"Copied: {f}")

print(f"\nSaved to: {OUTPUT_DIR}")

In [ ]:
# Show output files
total = 0
print("\nQuantized model files:")
for f in sorted(os.listdir(OUTPUT_DIR)):
    path = os.path.join(OUTPUT_DIR, f)
    if os.path.isfile(path):
        size = os.path.getsize(path) / 1024**2
        total += size
        print(f"  {f}: {size:.1f} MB")
print(f"\nTotal: {total:.1f} MB ({total/1024:.2f} GB)")

## Step 7: Verify Quantized Model

In [ ]:
# Clear memory
del model
torch.cuda.empty_cache()
gc.collect()

print("Loading quantized model...")
quantized_model = AutoModelForCausalLM.from_pretrained(
    OUTPUT_DIR,
    trust_remote_code=True,
    device_map="cuda"
)
print(f"Loaded! GPU mem: {torch.cuda.memory_allocated()/1024**3:.2f} GB")

In [ ]:
# Quick inference test
test_input = tokenizer("What text is in this image?", return_tensors="pt").to("cuda")
with torch.no_grad():
    output = quantized_model.generate(**test_input, max_new_tokens=20, do_sample=False)
print(f"Output: {tokenizer.decode(output[0], skip_special_tokens=True)}")
print("\n✓ Quantized model working!")

## Step 8: Create ZIP for Download (Optional)

In [ ]:
!cd /kaggle/working && zip -r dots_ocr_awq_4bit.zip dots_ocr_awq_4bit/

zip_size = os.path.getsize("/kaggle/working/dots_ocr_awq_4bit.zip") / 1024**3
print(f"\n✓ Created: dots_ocr_awq_4bit.zip ({zip_size:.2f} GB)")
print("\nDownload from Output tab →")

## Step 9: Push to HuggingFace Hub 🚀

Upload the quantized model to HuggingFace so you can use it directly with `transformers`.

**Requirements:**
1. Create a HuggingFace account at https://huggingface.co
2. Create an access token at https://huggingface.co/settings/tokens (with **write** permissions)
3. Add your token as a Kaggle secret named `HF_TOKEN`:
   - Go to Add-ons → Secrets
   - Add secret with Label: `HF_TOKEN`, Value: your token

In [ ]:
from huggingface_hub import HfApi, login
from kaggle_secrets import UserSecretsClient

# Get HuggingFace token from Kaggle secrets
try:
    user_secrets = UserSecretsClient()
    hf_token = user_secrets.get_secret("HF_TOKEN")
    print("✓ HuggingFace token loaded from Kaggle secrets")
except Exception as e:
    print("❌ Could not load HF_TOKEN from Kaggle secrets")
    print("Please add your HuggingFace token as a Kaggle secret named 'HF_TOKEN'")
    print("Go to: Add-ons → Secrets → Add secret")
    raise e

# Login to HuggingFace
login(token=hf_token)
print("✓ Logged in to HuggingFace")

In [ ]:
# ⚠️ CHANGE THIS to your HuggingFace username/repo-name
HF_REPO_ID = "sugam24/dots-ocr-awq-4bit"  # e.g., "sugam/dots-ocr-awq-4bit"

print(f"Will upload to: https://huggingface.co/{HF_REPO_ID}")
print("\n⚠️ Make sure to change HF_REPO_ID above to your username!")

In [ ]:
# Create a README for the model
readme_content = f"""---
license: apache-2.0
base_model: rednote-hilab/dots.ocr
tags:
  - ocr
  - vision
  - quantized
  - awq
  - 4bit
library_name: transformers
pipeline_tag: image-to-text
---

# dots.ocr AWQ 4-bit Quantized

This is a 4-bit AWQ quantized version of [rednote-hilab/dots.ocr](https://huggingface.co/rednote-hilab/dots.ocr).

## Model Details

- **Base Model**: rednote-hilab/dots.ocr
- **Quantization**: W4A16 (4-bit weights, 16-bit activations)
- **Method**: llm-compressor
- **Size**: ~1.5GB (reduced from ~6GB)

## Usage

```python
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoProcessor

model = AutoModelForCausalLM.from_pretrained(
    "{HF_REPO_ID}",
    trust_remote_code=True,
    device_map="cuda"
)
tokenizer = AutoTokenizer.from_pretrained("{HF_REPO_ID}", trust_remote_code=True)
processor = AutoProcessor.from_pretrained("{HF_REPO_ID}", trust_remote_code=True)
```

## License

Same as the base model (Apache 2.0).
"""

with open(os.path.join(OUTPUT_DIR, "README.md"), "w") as f:
    f.write(readme_content)

print("✓ Created README.md")

In [ ]:
# Upload to HuggingFace Hub
api = HfApi()

print(f"Uploading to {HF_REPO_ID}...")
print("This may take 5-15 minutes depending on your connection.")
print()

api.create_repo(repo_id=HF_REPO_ID, exist_ok=True, private=False)

api.upload_folder(
    folder_path=OUTPUT_DIR,
    repo_id=HF_REPO_ID,
    commit_message="Upload AWQ 4-bit quantized dots.ocr model",
)

print("="*50)
print("✓ Upload Complete!")
print("="*50)
print(f"\nModel available at: https://huggingface.co/{HF_REPO_ID}")
print(f"\nUsage:")
print(f'  model = AutoModelForCausalLM.from_pretrained("{HF_REPO_ID}", trust_remote_code=True, device_map="cuda")')

## Done! 🎉

Your quantized model is now on HuggingFace! Use it anywhere with:

```python
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoProcessor

model = AutoModelForCausalLM.from_pretrained(
    "YOUR_USERNAME/dots-ocr-awq-4bit",
    trust_remote_code=True,
    device_map="cuda"
)
tokenizer = AutoTokenizer.from_pretrained("YOUR_USERNAME/dots-ocr-awq-4bit", trust_remote_code=True)
processor = AutoProcessor.from_pretrained("YOUR_USERNAME/dots-ocr-awq-4bit", trust_remote_code=True)
```

## Step 10: Inference with Uploaded Model 🔍

Now let's test our uploaded quantized model by running OCR on a real image!

In [ ]:
# Install required packages for inference
print("Installing compressed-tensors for quantized model loading...")
!pip install compressed-tensors --quiet

print("✅ Installation complete!")
print("⚠️  IMPORTANT: You may need to restart the kernel after installation")
print("   If you get import errors in the next cell, restart the kernel and re-run from this cell")

# Test if the import works
try:
    import compressed_tensors
    print("✓ compressed-tensors successfully imported!")
except ImportError as e:
    print(f"❌ Import failed: {e}")
    print("💡 Please restart the kernel and re-run this cell")

### 📋 Inference Setup Instructions

**IMPORTANT: Follow these steps in order:**

1. **Run the installation cell below** (to install compressed-tensors)
2. **If you get import errors later**, restart the kernel:
   - Kernel → Restart Kernel 
   - Then re-run from the installation cell
3. **Run the model loading cell** 
4. **Continue with image loading and inference**

In [ ]:
# Import required libraries
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoProcessor
import torch
import gc
import os

# Clear previous model from memory
try:
    del quantized_model
    torch.cuda.empty_cache()
    gc.collect()
    print("✓ Cleared previous model from memory")
except:
    pass

# Load the quantized model from HuggingFace
print("Loading quantized model from HuggingFace...")
model_inference = AutoModelForCausalLM.from_pretrained(
    "sugam24/dots-ocr-awq-4bit",
    trust_remote_code=True,
    device_map="cuda",
    torch_dtype=torch.float16,
    attn_implementation="eager"  # Use eager attention to avoid mixed precision issues
)

# Note: Quantized models are already in the correct dtype, no need to call .half()

tokenizer_inference = AutoTokenizer.from_pretrained("sugam24/dots-ocr-awq-4bit", trust_remote_code=True)
processor_inference = AutoProcessor.from_pretrained("sugam24/dots-ocr-awq-4bit", trust_remote_code=True)

print(f"✓ Model loaded from HuggingFace! GPU mem: {torch.cuda.memory_allocated()/1024**3:.2f} GB")

In [ ]:
# Load and display the image
from PIL import Image
import matplotlib.pyplot as plt

image_path = "/kaggle/input/images/doc1.jpeg"

# Check if the image exists
if os.path.exists(image_path):
    image = Image.open(image_path)
    print(f"✓ Image loaded: {image.size} ({image.mode})")
    
    # Display the image
    plt.figure(figsize=(10, 8))
    plt.imshow(image)
    plt.axis('off')
    plt.title("Input Image for OCR")
    plt.show()
else:
    print(f"❌ Image not found at: {image_path}")
    print("Available files in /kaggle/input/:")
    for root, dirs, files in os.walk("/kaggle/input/"):
        for file in files:
            print(f"  {os.path.join(root, file)}")

In [ ]:
# Run OCR inference
print("Running OCR inference...")

# Prepare the input - dots.ocr uses a simpler approach
prompt = "What text is shown in this image?"

# Process image and text together using the dots.ocr processor
inputs = processor_inference(
    text=prompt,
    images=image,
    return_tensors="pt",
    padding=True
)

# Ensure all inputs are float16 to match the model (more comprehensive)
print("Converting input dtypes...")
for key in inputs:
    if isinstance(inputs[key], torch.Tensor):
        print(f"  {key}: {inputs[key].dtype} -> float16")
        # Force conversion to float16 regardless of current dtype
        inputs[key] = inputs[key].to(torch.float16)
    inputs[key] = inputs[key].to("cuda")

print(f"✓ Input processed. Keys: {list(inputs.keys())}")

# Generate the response with explicit dtype management
print("Generating response...")
with torch.autocast(device_type='cuda', dtype=torch.float16):  # Force float16 autocast
    with torch.no_grad():
        generated_ids = model_inference.generate(
            **inputs,
            max_new_tokens=2048,
            do_sample=False,
            temperature=0.0,
            pad_token_id=tokenizer_inference.eos_token_id,
            use_cache=True
        )

# Decode the response
generated_ids_trimmed = [
    out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]

output_text = processor_inference.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)[0]

print("✓ OCR inference completed!")
print("\n" + "="*50)
print("EXTRACTED TEXT:")
print("="*50)
print(output_text)
print("="*50)

In [ ]:
# Save results to markdown file
from datetime import datetime

# Create markdown content
markdown_content = f"""# OCR Results - dots.ocr AWQ 4-bit

## Inference Details
- **Model**: sugam24/dots-ocr-awq-4bit
- **Quantization**: W4A16 (4-bit weights, 16-bit activations)
- **Input Image**: {image_path}
- **Image Size**: {image.size}
- **Timestamp**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Prompt
```
{prompt}
```

## Extracted Text

```
{output_text}
```

---
*Generated using the AWQ 4-bit quantized version of dots.ocr model*
"""

# Save to file
output_md_path = "/kaggle/working/ocr_results.md"
with open(output_md_path, 'w', encoding='utf-8') as f:
    f.write(markdown_content)

print(f"✅ Results saved to: {output_md_path}")
print(f"📄 File size: {os.path.getsize(output_md_path)} bytes")

# Also display the markdown content
print("\n" + "="*50)
print("MARKDOWN CONTENT:")
print("="*50)
print(markdown_content)

In [ ]:
# Cleanup and summary
torch.cuda.empty_cache()
gc.collect()

print("🎯 INFERENCE COMPLETE!")
print("\n📋 Summary:")
print(f"   • Model: sugam24/dots-ocr-awq-4bit")
print(f"   • Image: {image_path}")
print(f"   • Results: {output_md_path}")
print(f"   • GPU Memory: {torch.cuda.memory_allocated()/1024**3:.2f} GB")

print("\n📁 Available outputs:")
print("   • OCR results: /kaggle/working/ocr_results.md")
print("   • Quantized model: /kaggle/working/dots_ocr_awq_4bit/")
print("   • Model ZIP: /kaggle/working/dots_ocr_awq_4bit.zip")

print("\n✨ The quantized model is now ready for production use!")
print("   You can download the markdown file from the Output tab.")